In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


1. Load Data

In [12]:
import pandas as pd

train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')


2. Combine Train and Test for Uniform Processing

In [13]:
import numpy as np
test_df['SalePrice'] = np.nan

combined = pd.concat([train_df, test_df], ignore_index=True)


3. Missing Value Treatment

In [14]:
# Check missing values
missing = combined.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print(missing)


PoolQC          2909
MiscFeature     2814
Alley           2721
Fence           2348
MasVnrType      1766
SalePrice       1459
FireplaceQu     1420
LotFrontage      486
GarageCond       159
GarageYrBlt      159
GarageFinish     159
GarageQual       159
GarageType       157
BsmtExposure      82
BsmtCond          82
BsmtQual          81
BsmtFinType2      80
BsmtFinType1      79
MasVnrArea        23
MSZoning           4
Functional         2
BsmtHalfBath       2
BsmtFullBath       2
Utilities          2
KitchenQual        1
TotalBsmtSF        1
BsmtUnfSF          1
GarageCars         1
GarageArea         1
BsmtFinSF2         1
BsmtFinSF1         1
Exterior2nd        1
Exterior1st        1
SaleType           1
Electrical         1
dtype: int64


Handle Missing:
Categorical: Fill with mode or "None"

Numerical: Fill with median or 0


In [15]:

combined['PoolQC'] = combined['PoolQC'].fillna("None")
combined['LotFrontage'] = combined['LotFrontage'].fillna(combined['LotFrontage'].median())


 Convert Categorical to Category dtype

In [16]:
categorical_cols = combined.select_dtypes(include='object').columns
for col in categorical_cols:
    combined[col] = combined[col].astype('category')


5. Label Encoding / One-Hot Encoding

In [17]:
combined = pd.get_dummies(combined, drop_first=True)


6. Split Train and Test Again

In [18]:
train = combined[combined['SalePrice'].notnull()]
test = combined[combined['SalePrice'].isnull()].drop('SalePrice', axis=1)


7. Separate Features and Target

In [19]:
X = train.drop(['SalePrice', 'Id'], axis=1)
y = train['SalePrice']
test_ids = test_df['Id']


8. Feature Engineering

TotalSF = TotalBsmtSF + 1stFlrSF + 2ndFlrSF

TotalBath = FullBath + HalfBath*0.5

Age = YrSold - YearBuilt

In [21]:
combined['TotalSF'] = combined['TotalBsmtSF'] + combined['1stFlrSF'] + combined['2ndFlrSF']
combined['Age'] = combined['YrSold'] - combined['YearBuilt']
